# Ungraded Lab: Building a Custom Dense Layer

In this lab, we'll walk through how to create a custom layer that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) class. Unlike simple Lambda layers you did previously, the custom layer here will contain weights that can be updated during training.

## Imports

In [1]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import numpy as np

## Custom Layer with weights

To make custom layer that is trainable, we need to define a class that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) base class from Keras. The Python syntax is shown below in the class declaration. This class requires three functions: `__init__()`, `build()` and `call()`. These ensure that our custom layer has a *state* and *computation* that can be accessed during training or inference.

In [2]:
# inherit from this base class
from tensorflow.keras.layers import Layer

class SimpleDense(Layer):

    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        print("Constructor Called")
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        print("build() called")
        print(f"Input shape = {np.array(input_shape)}")
        print(f"Units = {self.units}")
        # initialize the weights
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        print("call() called")
        return tf.matmul(inputs, self.w) + self.b

Now we can use our custom layer like below:

In [3]:
# declare an instance of the class
my_dense = SimpleDense(units=1)# __init__() called

# define an input and feed into the layer
x = tf.ones((3, 4))
y = my_dense(x)

# parameters of the base Layer class like `variables` can be used
print(my_dense.variables)

Constructor Called
build() called
Input shape = [3 4]
Units = 1
call() called
[<tf.Variable 'simple_dense/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[-0.05739428],
       [-0.01745657],
       [ 0.0080457 ],
       [ 0.01801712]], dtype=float32)>, <tf.Variable 'simple_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


Let's then try using it in simple network:

In [4]:
# define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)


# use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])

# configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500,verbose=1)

# perform inference
print(model.predict([10.0]))

# see the updated state of the variables
print(my_layer.variables)
print(model.summary())

Constructor Called
build() called
Input shape = [None 1]
Units = 1
call() called
Train on 6 samples
Epoch 1/500
call() called
call() called
6/6 [==============================] - 0s 24ms/sample - loss: 14.5300
Epoch 2/500
6/6 [==============================] - 0s 224us/sample - loss: 11.6691
Epoch 3/500
6/6 [==============================] - 0s 160us/sample - loss: 9.4134
Epoch 4/500
6/6 [==============================] - 0s 157us/sample - loss: 7.6340
Epoch 5/500
6/6 [==============================] - 0s 142us/sample - loss: 6.2293
Epoch 6/500
6/6 [==============================] - 0s 168us/sample - loss: 5.1196
Epoch 7/500
6/6 [==============================] - 0s 147us/sample - loss: 4.2420
Epoch 8/500
6/6 [==============================] - 0s 144us/sample - loss: 3.5472
Epoch 9/500
6/6 [==============================] - 0s 141us/sample - loss: 2.9962
Epoch 10/500
6/6 [==============================] - 0s 140us/sample - loss: 2.5586
Epoch 11/500
6/6 [==============================] 

In [5]:
try:
    my_layer = SimpleDense(units=1)
    model = tf.keras.Sequential([my_layer])
    print(model.summary())
except ValueError as e:
    print("Error!!! ", e)

Constructor Called
Error!!!  This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.
